In [44]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem

In [45]:
import random, copy

def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else cano(smiles)

# choose carboxylic acid (tmp)
acid_smi = 'CC(C)CCC(O)=O'
acid = Chem.MolFromSmiles(acid_smi)

# choose alcohol (tmp)
alcohol_smi = 'CC(C)CO'

# choose amine (tmp)
amine_smi = 'CC(C)CN'

In [46]:
## Carboxylic acid

DIAZO = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[C:3]=[N+]=[N-]>>[C:1](=O)[O:2][C:3]')
RED = AllChem.ReactionFromSmarts('[C:1](=O)[O:2]>>[C:1][O:2]')
RLI = AllChem.ReactionFromSmarts('[C:1](=[O:2])O.[C:3][Li]>>[C:1](=[O:2])[C:3]')
CHLORI = AllChem.ReactionFromSmarts('[C:1](=[O:2])O>>[C:1](=[O:2])Cl')
HVZ = AllChem.ReactionFromSmarts('[CH2:1][C:2](=O)O>>[C:1](Br)[C:2](=O)O')
ESTER = AllChem.ReactionFromSmarts('[C:1](=[O:2])[O:3].[O:4]>>[C:1](=[O:2])[O:4]')

acid_reactions_list = [
    
    ('DIAZO', DIAZO, ['C=[N+]=[N-]'], ['']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]', 'B.C1CCOC1']),
    ('RLI', RLI, ['[Li]C', '[Li]CC', '[Li]CCC', '[Li]CCCC', '[Li]c1ccccc1'], ['']),
    ('CHLORI', CHLORI, [''], ['ClS(Cl)=O', 'ClC(=O)C(Cl)=O']),
    ('HVZ', HVZ, [''], ['BrBr.BrP(Br)Br']),
    ('ESTER', ESTER, [''], ['Cl', 'OS(O)(=O)=O']),
    
]

# general reactions
for reaction in acid_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ESTER':
            reagent = alcohol_smi
        if reagent == '': products = reaction[1].RunReactants((acid,))
        else: products = reaction[1].RunReactants((acid, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(acid_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

DIAZO
CC(C)CCC(=O)O.C=[N+]=[N-]>>COC(=O)CCC(C)C
RED
CC(C)CCC(=O)O>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(C)CCCO
CC(C)CCC(=O)O>B.C1CCOC1>CC(C)CCCO
RLI
CC(C)CCC(=O)O.[Li]C>>CC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CC>>CCC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CCC>>CCCC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CCCC>>CCCCC(=O)CCC(C)C
CHLORI
CC(C)CCC(=O)O>O=S(Cl)Cl>CC(C)CCC(=O)Cl
CC(C)CCC(=O)O>O=C(Cl)C(=O)Cl>CC(C)CCC(=O)Cl
HVZ
CC(C)CCC(=O)O>BrBr.BrP(Br)Br>CC(C)CC(Br)C(=O)O
ESTER
CC(C)CCC(=O)O.CC(C)CO>Cl>CC(C)CCC(=O)OCC(C)C
CC(C)CCC(=O)O.CC(C)CO>O=S(=O)(O)O>CC(C)CCC(=O)OCC(C)C


In [47]:
chloride = CHLORI.RunReactants((acid,))[0][0]
chloride_smi = Chem.MolToSmiles(chloride)

In [48]:
ANHYD = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[C:3](=O)[O:4]>>[C:1](=O)[O:4][C:3](=O)')
ESTER = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[O:3]>>[C:1](=O)[O:3]')
AMIDE = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[N:3]>>[C:1](=O)[N:3]')
CARBO = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[O:3]>>[C:1](=O)[O:3]')

METAL = AllChem.ReactionFromSmarts('[C:1](=[O:2])[Cl:3].[C,c:4][Mg+:5]>>[C:1]([*:4])([*:4])[O:2]')
RED = AllChem.ReactionFromSmarts('[C:1](=[O:2])[Cl:3]>>[C:1](=[O:2])')

chloride_reactions_list = [
    
    ('ANHYD', ANHYD, [''], ['']),
    ('ESTER', ESTER, [''], ['']),
    ('AMIDE', AMIDE, [''], ['']),
    ('CARBO', CARBO, ['O'], ['']),
    
    ('METAL', METAL, ['[Br-].[Mg+]C', '[Br-].[Mg+]CC', '[Br-].[Mg+]CCC', \
                      '[Br-].[Mg+]CCCC', '[Br-].[Mg+]c1ccccc1'], ['']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]']),
    
]


# general reactions
for reaction in chloride_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ANHYD':
            reagent = acid_smi
        elif reaction[0] == 'ESTER':
            reagent = alcohol_smi
        elif reaction[0] == 'AMIDE':
            reagent = amine_smi
        if reagent == '': products = reaction[1].RunReactants((chloride,))
        else: products = reaction[1].RunReactants((chloride, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(chloride_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

ANHYD
CC(C)CCC(=O)Cl.CC(C)CCC(=O)O>>CC(C)CCC(=O)OC(=O)CCC(C)C
ESTER
CC(C)CCC(=O)Cl.CC(C)CO>>CC(C)CCC(=O)OCC(C)C
AMIDE
CC(C)CCC(=O)Cl.CC(C)CN>>CC(C)CCC(=O)NCC(C)C
CARBO
CC(C)CCC(=O)Cl.O>>CC(C)CCC(=O)O
METAL
CC(C)CCC(=O)Cl.C[Mg+].[Br-]>>CC(C)CCC(C)(C)O
CC(C)CCC(=O)Cl.CC[Mg+].[Br-]>>CCC(O)(CC)CCC(C)C
CC(C)CCC(=O)Cl.CCC[Mg+].[Br-]>>CCCC(O)(CCC)CCC(C)C
CC(C)CCC(=O)Cl.CCCC[Mg+].[Br-]>>CCCCC(O)(CCCC)CCC(C)C
CC(C)CCC(=O)Cl.[Br-].[Mg+]c1ccccc1>>CC(C)CCC(O)(c1ccccc1)c1ccccc1
RED
CC(C)CCC(=O)Cl>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(C)CCC=O


In [49]:
anhyd = ANHYD.RunReactants((chloride, Chem.MolFromSmiles(acid_smi,)))[0][0]
anhyd_smi = Chem.MolToSmiles(anhyd)

In [50]:
ESTER = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[O:3]>>[C:1](=O)[O:3]')
AMIDE = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[N:3]>>[C:1](=O)[N:3]')
CARBO = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[O:3]>>[C:1](=O)[O:3]')

anhydride_reactions_list = [
    
    ('ESTER', ESTER, [''], ['Cl', 'OS(O)(=O)=O']),
    ('AMIDE', AMIDE, [''], ['']),
    ('CARBO', CARBO, ['O'], ['']),
    
]

# general reactions
for reaction in anhydride_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ESTER':
            reagent = alcohol_smi
        elif reaction[0] == 'AMIDE':
            reagent = amine_smi
        if reagent == '': products = reaction[1].RunReactants((anhyd,))
        else: products = reaction[1].RunReactants((anhyd, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(anhyd_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

ESTER
CC(C)CCC(=O)OC(=O)CCC(C)C.CC(C)CO>Cl>CC(C)CCC(=O)OCC(C)C
CC(C)CCC(=O)OC(=O)CCC(C)C.CC(C)CO>O=S(=O)(O)O>CC(C)CCC(=O)OCC(C)C
AMIDE
CC(C)CCC(=O)OC(=O)CCC(C)C.CC(C)CN>>CC(C)CCC(=O)NCC(C)C
CARBO
CC(C)CCC(=O)OC(=O)CCC(C)C.O>>CC(C)CCC(=O)O


In [52]:
# choose ester (tmp)
ester_smi = 'CC(C)CCC(=O)OCC(C)C'
ester = Chem.MolFromSmiles(ester_smi)

_DEP = AllChem.ReactionFromSmarts('[O:1]>>[O-:1]')
alkoxide_smi = Chem.MolToSmiles(_DEP.RunReactants((Chem.MolFromSmiles(alcohol_smi),))[0][0])

In [60]:
HYDRO = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[O:3]>>[C:1](=O)[O:3].[O:2]')
ESTER = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[O:3]>>[C:1](=O)[O:3]')
AMIDE = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[N:3]>>[C:1](=O)[N:3]')

RED = AllChem.ReactionFromSmarts('[C:1](=O)[O:2]>>[C:1]O')
METAL = AllChem.ReactionFromSmarts('[C:1](=[O:2])[O:3].[C,c:4][Mg+:5]>>[C:1]([*:4])([*:4])[O:2]')

ester_reactions_list = [
    
    ('HYDRO', HYDRO, ['O'], ['Cl', 'OS(O)(=O)=O', '[Na+].[OH-]']),
    ('ESTER', ESTER, [''], ['']),
    ('AMIDE', AMIDE, [''], ['']),
    
    ('METAL', METAL, ['[Br-].[Mg+]C', '[Br-].[Mg+]CC', '[Br-].[Mg+]CCC', \
                      '[Br-].[Mg+]CCCC', '[Br-].[Mg+]c1ccccc1'], ['']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]']),
    
]


# general reactions
for reaction in ester_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ESTER':
            reagent = alcohol_smi
        elif reaction[0] == 'AMIDE':
            reagent = amine_smi
        if reagent == '': products = reaction[1].RunReactants((ester,))
        else: products = reaction[1].RunReactants((ester, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        if reaction[0] == 'ESTER':
            r_ester = [('[Na+].'+alkoxide_smi, alcohol_smi), (alcohol_smi, 'Cl'), (alcohol_smi, 'OS(O)(=O)=O')]
            for r in r_ester:
                print(cano(ester_smi) + block('.', r[0]) + '>' + cano(r[1]) + '>' + '.'.join(product_smi))
        else:
            for sub_reagent in reaction[3]:
                print(cano(ester_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

HYDRO
CC(C)CCC(=O)OCC(C)C.O>Cl>CC(C)CCC(=O)O.CC(C)CO
CC(C)CCC(=O)OCC(C)C.O>O=S(=O)(O)O>CC(C)CCC(=O)O.CC(C)CO
CC(C)CCC(=O)OCC(C)C.O>[Na+].[OH-]>CC(C)CCC(=O)O.CC(C)CO
ESTER
CC(C)CCC(=O)OCC(C)C.CC(C)C[O-].[Na+]>CC(C)CO>CC(C)CCC(=O)OCC(C)C
CC(C)CCC(=O)OCC(C)C.CC(C)CO>Cl>CC(C)CCC(=O)OCC(C)C
CC(C)CCC(=O)OCC(C)C.CC(C)CO>O=S(=O)(O)O>CC(C)CCC(=O)OCC(C)C
AMIDE
CC(C)CCC(=O)OCC(C)C.CC(C)CN>>CC(C)CCC(=O)NCC(C)C
METAL
CC(C)CCC(=O)OCC(C)C.C[Mg+].[Br-]>>CC(C)CCC(C)(C)O
CC(C)CCC(=O)OCC(C)C.CC[Mg+].[Br-]>>CCC(O)(CC)CCC(C)C
CC(C)CCC(=O)OCC(C)C.CCC[Mg+].[Br-]>>CCCC(O)(CCC)CCC(C)C
CC(C)CCC(=O)OCC(C)C.CCCC[Mg+].[Br-]>>CCCCC(O)(CCCC)CCC(C)C
CC(C)CCC(=O)OCC(C)C.[Br-].[Mg+]c1ccccc1>>CC(C)CCC(O)(c1ccccc1)c1ccccc1
RED
CC(C)CCC(=O)OCC(C)C>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(C)CCCO


In [79]:
# choose amide (tmp)
amide_smi = 'CC(C)CCC(=O)NCC(C)C'
amide = Chem.MolFromSmiles(amide_smi)

In [80]:
HYD = AllChem.ReactionFromSmarts('[C:1](=O)[N:2].[O:3]>>[C:1](=O)[O:3].[N:2]')
RED = AllChem.ReactionFromSmarts('[C:1](=[O:2])[N:3]>>[C:1][N:3]')

amide_reactions_list = [
    
    ('HYD', HYD, ['O'], ['Cl', 'OS(O)(=O)=O', '[Na+].[OH-]']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]']),
    
]


# general reactions
for reaction in amide_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reagent == '': products = reaction[1].RunReactants((amide,))
        else: products = reaction[1].RunReactants((amide, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(amide_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

HYD
CC(C)CCC(=O)NCC(C)C.O>Cl>CC(C)CCC(=O)O.CC(C)CN
CC(C)CCC(=O)NCC(C)C.O>O=S(=O)(O)O>CC(C)CCC(=O)O.CC(C)CN
CC(C)CCC(=O)NCC(C)C.O>[Na+].[OH-]>CC(C)CCC(=O)O.CC(C)CN
RED
CC(C)CCC(=O)NCC(C)C>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(C)CCCNCC(C)C


In [81]:
# primary amine (tmp)
amide_1_smi = 'CC(C)(C)CC(C)CC(=O)N'
amide_1 = Chem.MolFromSmiles(amide_1_smi)

In [82]:
# primary amide
HOFMANN = AllChem.ReactionFromSmarts('[C:1]C(=O)N>>[C:1]N')
DEHYD = AllChem.ReactionFromSmarts('[C:1](=O)N>>[C:1]#N')

amide_1_reactions_list = [
    
    ('HOFMANN', HOFMANN, [''], ['BrBr.[Na+].[OH-]']),
    ('DEHYD', DEHYD, [''], ['ClP(Cl)(Cl)=O', 'O=P12OP3(=O)OP(=O)(O1)OP(=O)(O2)O3']),
    
]


# general reactions
for reaction in amide_1_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reagent == '': products = reaction[1].RunReactants((amide_1,))
        else: products = reaction[1].RunReactants((amide_1, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(amide_1_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

HOFMANN
CC(CC(N)=O)CC(C)(C)C>BrBr.[Na+].[OH-]>CC(CN)CC(C)(C)C
DEHYD
CC(CC(N)=O)CC(C)(C)C>O=P(Cl)(Cl)Cl>CC(CC#N)CC(C)(C)C
CC(CC(N)=O)CC(C)(C)C>O=P12OP3(=O)OP(=O)(O1)OP(=O)(O2)O3>CC(CC#N)CC(C)(C)C


In [91]:
# nitrile (tmp)
nitrile_smi = 'CC(C#N)CC(C)(C)C'
nitrile = Chem.MolFromSmiles(nitrile_smi)

In [92]:
HYD = AllChem.ReactionFromSmarts('[C:1]#N.O>>[C:1](=O)O')
RED = AllChem.ReactionFromSmarts('[C:1]#[N:2]>>[C:1][N:2]')
METAL = AllChem.ReactionFromSmarts('[C:1]#N.[C,c:2][Mg+]>>[C:1](=O)[*:2]')

nitrile_reactions_list = [
    
    ('HYD', HYD, ['O'], ['Cl', 'OS(O)(=O)=O', '[Na+].[OH-]']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]']),
    ('METAL', METAL, ['[Br-].[Mg+]C', '[Br-].[Mg+]CC', '[Br-].[Mg+]CCC', \
                      '[Br-].[Mg+]CCCC', '[Br-].[Mg+]c1ccccc1'], ['']),
    
]

# general reactions
for reaction in nitrile_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reagent == '': products = reaction[1].RunReactants((nitrile,))
        else: products = reaction[1].RunReactants((nitrile, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(nitrile_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

HYD
CC(C#N)CC(C)(C)C.O>Cl>CC(CC(C)(C)C)C(=O)O
CC(C#N)CC(C)(C)C.O>O=S(=O)(O)O>CC(CC(C)(C)C)C(=O)O
CC(C#N)CC(C)(C)C.O>[Na+].[OH-]>CC(CC(C)(C)C)C(=O)O
RED
CC(C#N)CC(C)(C)C>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(CN)CC(C)(C)C
METAL
CC(C#N)CC(C)(C)C.C[Mg+].[Br-]>>CC(=O)C(C)CC(C)(C)C
CC(C#N)CC(C)(C)C.CC[Mg+].[Br-]>>CCC(=O)C(C)CC(C)(C)C
CC(C#N)CC(C)(C)C.CCC[Mg+].[Br-]>>CCCC(=O)C(C)CC(C)(C)C
CC(C#N)CC(C)(C)C.CCCC[Mg+].[Br-]>>CCCCC(=O)C(C)CC(C)(C)C
CC(C#N)CC(C)(C)C.[Br-].[Mg+]c1ccccc1>>CC(CC(C)(C)C)C(=O)c1ccccc1
